<a href="https://colab.research.google.com/github/shrikant280304/FMML_PROJECTS_AND_LABS/blob/main/M5_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
from tqdm import tqdm

In [ ]:
with open("/content/fingerDataSet.pickle", "rb") as infile:
    dat = pickle.load(infile, encoding="bytes")

FileNotFoundError: [Errno 2] No such file or directory: '/content/fingerDataSet.pickle'

In [ ]:
import numpy as np
from PIL import Image, ImageDraw
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

In [ ]:
dat[0].keys()

In [ ]:
def highlightPoints(img, pts, color = "yellow"):
    draw = ImageDraw.Draw(img)
    for y, x in pts:
        draw.point((x, y), fill = color)

In [ ]:
imDat = dat[50]
for key, item in imDat.items():
    try:
        print(key, type(item), item.shape)
    except:
        print(key, type(item), item)

In [ ]:
imDat = dat[32]
img = Image.fromarray(imDat[b'croppedImage'])
img = img.convert(mode="RGB")
newFingerPoints = [(y - min(imDat[b"box"][:,0]), x - min(imDat[b"box"][: , 1])) for y, x in imDat[b"allFingerPoints"]]
# highlightPoints(img, [(173, 211), (192, 253)])
highlightPoints(img, newFingerPoints)
# highlightPoints(img, imDat[b"box"], "blue")


imshow(img)
plt.gcf().set_dpi(70)

In [ ]:
FILTER = 11

In [ ]:
class PointDat:
    def __init__(self, imgAr, isFinger : bool, coor = None, idx = None):
        self.imgAr = imgAr
        self.isFinger = isFinger
        self.idx = idx
        self.coor = coor

    def __str__(self):
        return "Coors {} on idx {}".format(self.coor, self.idx)

    def data(self):
        return self.imgAr.reshape(-1)

    def getPointGrid(self):
        return self.imgAr

    def getBoolean(self):
        return self.isFinger

    def getCoors(self):
        return self.coor

In [ ]:

def highlightPoints(img, pts, color = "yellow"):
    draw = ImageDraw.Draw(img)
    for y, x in pts:
        draw.point((x, y), fill = color)

class ImgDat:
    def __init__(self, dat, idx = None):
        self.idx = idx
        self.fullImage = dat[b"image"]
        self.box = dat[b"box"]
        self.topLeft = (int(min(dat[b"box"][:, 0])), int(min(dat[b"box"][:, 1])))
        self.bottomRight = (int(max(dat[b"box"][:, 0])), int(max(dat[b"box"][:, 1])))

        self.bitImage = dat[b"image1bit"][self.topLeft[0]:self.bottomRight[0], self.topLeft[1]:self.bottomRight[1]]
        self.numFingers = dat[b"numFingers"]
        self.boundImage = dat[b"croppedImage"]

        self.fingerPoints = [(p1 - self.topLeft[0], p2 - self.topLeft[1]) for p1, p2 in dat[b"allFingerPoints"]]

    def showImage(self):
        img = Image.fromarray(self.boundImage)
        img = img.convert(mode = "RGB")
        return img

    def showFingers(self):
        COLS = ["orange", "pink", "lime"]
        img = Image.fromarray(self.boundImage)
        img = img.convert(mode = "RGB")
        highlightPoints(img, self.fingerPoints, COLS[self.numFingers - 1])
        return img

    def showFullImage(self):
        img = Image.fromarray(self.fullImage)
        img = img.convert(mode = "RGB")
        highlightPoints(img, self.box, "yellow")
        return img

    def makePointsDataset(self):
        F = FILTER//2

        collection = []
        for r in range(F, self.bitImage.shape[0] - F):
            for c in range(F, self.bitImage.shape[1] - F):
#                 collection.append([self.bitImage[r-F:r+F, c-F:c+F], (r, c) in self.fingerPoints, (r, c)])
                collection.append(PointDat(self.bitImage[r - F : r + F+1, c - F: c + F+1], (r, c) in self.fingerPoints, (r, c),\
                                            self.idx))

        return np.array(collection, dtype=object)



In [ ]:
sampleIndices = [0, 10, 52, 30, 40]
fig, axs = plt.subplots(5, 3, figsize=(8, 16), gridspec_kw={'width_ratios': [3, 1, 1]})
for r in range(5):
    p = ImgDat(dat[sampleIndices[r]], sampleIndices[r])
    axs[r][0].imshow(p.showFullImage())
    axs[r][1].imshow(p.showImage())
    axs[r][2].imshow(p.showFingers())
fig.set_dpi(100)

In [ ]:
p = ImgDat(dat[52], 52)
d = p.makePointsDataset()

In [ ]:
posPoints = [point for point in d if point.getBoolean()]
negPoints = [point for point in d if not point.getBoolean()]


### Positive Datapoints for grids that are finger points

In [ ]:
np.random.seed(0)
fig, axs = plt.subplots(3, 5, figsize=(15, 9))
for ax in axs:
    for a in ax:
        point = np.random.choice(posPoints)
        a.imshow(point.getPointGrid())
        a.title.set_text(str(point))
        a.set_xticks([FILTER//2])
        a.set_yticks([FILTER//2])

### Negative Datapoints for grids that are not finger points

In [ ]:
np.random.seed(0)
fig, axs = plt.subplots(3, 5, figsize=(15, 9))
for ax in axs:
    for a in ax:
        point = np.random.choice(negPoints)
        a.imshow(point.getPointGrid())
        a.title.set_text(str(point))
        a.set_xticks([FILTER//2])
        a.set_yticks([FILTER//2])

### Creating a method to cluster the points together and predict the number of fingers based on the number of clusters

In [ ]:
def singleton(cls, *args, **kw):
    instances = {}
    def _singleton(*args, **kw):
        if cls not in instances:
            instances[cls] = cls(*args, **kw)
        return instances[cls]
    return _singleton

@singleton
class fingersFromPoints:

    def findAdjacent(self, coordinates, coordinate, cluster):
        distances = np.sqrt(np.sum((coordinates-coordinate)**2,1))
        newCoordIndices = np.where(np.logical_and(distances<1.3, distances >0))[0]

        for newCoordIndex in newCoordIndices:
            if newCoordIndex not in cluster:
                cluster.append(newCoordIndex)
                self.findAdjacent(coordinates, coordinates[newCoordIndex, :], cluster)

    def findClusters(self, coordinates):
        allIndices = np.arange(len(coordinates))
        clusters = []

        while len(allIndices) > 0:

            cluster = [allIndices[0]]
            self.findAdjacent(coordinates, coordinates[allIndices[0],:], cluster)
            allIndices = np.setdiff1d(allIndices, np.array((cluster)))
            clusters.append(cluster)

        return clusters

    def findClusterCoors(self, coors):

        clusters = self.findClusters(coors)
        newClus = []
        for c in clusters:
            temp = []
            for idx in c:
                temp.append(coors[idx])
            newClus.append(temp)
        return newClus

    def countFingers(self, clusters, minFingerSize = 15, fingerCutoff = 0.5):
        sizes = []
        for cluster in clusters:
            sizes.append(len(cluster))

        sizes = np.array(sizes)

        if len(sizes) > 0:
            if np.max(sizes) > minFingerSize:
                fingerCluster = np.argmax(sizes)
                numFingers = sum(sizes>fingerCutoff*sizes[fingerCluster])
            else:
                numFingers = 0
        else:
            numFingers = 0

        return numFingers

    def predictNumFingers(self, coors):
        newCoors = np.array(coors)
        clusters = self.findClusters(newCoors)
        #print(clusters)
        return self.countFingers(clusters)


### Checking if the clustering is working with a datapoint

In [ ]:
predObj = fingersFromPoints()

imgObj = ImgDat(dat[52], 52)

assert imgObj.numFingers == predObj.predictNumFingers(imgObj.fingerPoints), "Clustering has failed for the dataimage \
with index {}. Something is wrong! {} is being predicted instead of {})".format(imgObj.idx, predObj.predictNumFingers(imgObj.fingerPoints),\
                                                                               imgObj.numFingers)


### Checking the accuracy over all the datapoints

In [ ]:
cor, tot = 0, 0
incor = []
for idx, d in enumerate(dat):
    imgObj = ImgDat(d, idx)
    if imgObj.numFingers == predObj.predictNumFingers(imgObj.fingerPoints):
        cor += 1
    else:
        incor.append(imgObj)
    tot += 1

print("The Clustering is correct for: {:.2f} % of the datapoints' ground truth".format(cor * 100 / tot))

The Answer is wrong for:

In [ ]:
print(len(incor))

### Let us examine on why are two of the input images are clustered incorrectly

In [ ]:
fig, axs = plt.subplots(2, 3, figsize = (12, 10))
incorIndices = []
for idx, (ax, imObj) in enumerate(zip(axs, incor)):
    print("Incorrect for {}".format(imObj.idx))
    incorIndices.append(imObj.idx)
    ax[0].imshow(imObj.showImage())
    ax[1].imshow(imObj.showFingers())
    img = imObj.showImage()
    clusters = predObj.findClusterCoors(np.array(imObj.fingerPoints))
    COLS = ["green", "red", "blue", "yello", "orange", "pink"]
    for idx2, c in enumerate(clusters):
        highlightPoints(img, c, COLS[idx2])
    ax[2].imshow(img)

In [ ]:
allImg = [ImgDat(d, idx) for idx, d in enumerate(dat)]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
trainImg, testImg = train_test_split(allImg, test_size = 10, train_size = 44, random_state = 0)

In [ ]:
trainPoints = np.array([], dtype=object)
for imgObj in tqdm(trainImg):
    trainPoints = np.append(trainPoints, imgObj.makePointsDataset())

In [ ]:
X_train, y_train = np.array([p.data() for p in trainPoints]), np.array([p.getBoolean() for p in trainPoints])

In [ ]:
X_train.shape, y_train.shape

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state = 0, max_depth = 3)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
#Training Error in terms of points:
from sklearn.metrics import accuracy_score
print("{:.2f} % of the training points classified correctly".format(100*accuracy_score(clf.predict(X_train), y_train)))

### Now lets us try it on the 10 test images

In [ ]:
fig, axs = plt.subplots(10, 3, figsize = (9, 50))
for idx, imgObj in enumerate(testImg):
    testPoints = imgObj.makePointsDataset()
    axs[idx][0].imshow(imgObj.showImage())
    axs[idx][1].imshow(imgObj.showFingers())

    coors = []
    for point in testPoints:
        if clf.predict([point.data()])[0]:
            coors.append(point.getCoors())
    clusters = predObj.findClusterCoors(np.array(coors))
#     clusters = sorted(clusters, key = lambda c : len(c))
#     clusters = clusters[::-1]
#     clusters = clusters[:5]
    img = imgObj.showImage()
    COLS = ["green", "red", "blue", "yellow", "orange", "pink"]
    for idx2, c in enumerate(clusters):
#         pass
        highlightPoints(img, c, COLS[idx2%6])
    axs[idx][2].imshow(img)
    if predObj.predictNumFingers(coors) == imgObj.numFingers:
        print("Predicted correctly for idx : {}, the number: {}".format(imgObj.idx, imgObj.numFingers))
    else:
        print("Predicted incorrectly for idx : {}, the number: {}".format(imgObj.idx, imgObj.numFingers))
#     print("Num clusters seen: {}".format(len(clusters)))


In [ ]:
from sklearn import tree

In [ ]:
fig, ax = plt.subplots(figsize=(20, 7))
imgObj = tree.plot_tree(clf, filled=True, ax=ax, impurity=False, proportion=True, rounded=True, fontsize = 10)
plt.show()

Visualizing the decision maker:

In [ ]:
blank = np.ones((11, 11))
img = Image.fromarray(blank)
img = img.convert(mode="RGB")

imDraw = ImageDraw.Draw(img)
imDraw.point((12%11, 12//11), fill = "red")
imDraw.point((54%11, 54//11), fill = "red")
imDraw.point((60%11, 60//11), fill = "green")
plt.imshow(img)